# Update existing DSC

The [Quick Start](Quick_Start.html) example contains 3 module / module ensembles, `simulate`, `estimate` and `mse`. Suppose we now have a new computational approach for location estimate via the *Winsorized* mean. This can be implemented as `winsor.R`:

```r
  mean = psych::winsor.mean(x, trim = trim, na.rm = TRUE)
```
and be incoroperated to DSC as a module.

For `winsor.R` there is an additional parameter `trim` which does not exist in `mean` or `median`. Syntactically are two ways to proceed: either by adding it to the existing group of modules, or by adding a new block that configures this module. We will show here two flavors to update, both involving minimal changes to the DSC file and result in the same outcome. In practice one should always adopt flavor that is most logically clear and succinct.

## Add module to Existing Block
We change the existing block to the following:

```
mean, median, winsor: mean.R, median.R, winsor.R
    x: $x
    @winsor:
      trim: 0.1, 0.2
    $mean: mean
```

where we added in a new module `winsor` with executable `winsor.R`, along with an executable specific parameter specification via `@winsor` to assign two possible values to `trim` which essentially provides two versions of Winsorization for comparison.

We also added `winsor` to the `estimate` ensemble:

```
DSC:
    define:
      simulate: normal, t
      estimate: mean, median, winsor
```

To run the new DSC,

In [1]:
%cd ~/GIT/dsc2/vignettes/one_sample_location

/home/gaow/GIT/dsc2/vignettes/one_sample_location

In [2]:
! dsc settings_add_method_v1.dsc -c 30

INFO: Checking R library psych ...
INFO: Checking R library dscrutils@stephenslab/dsc2/dscrutils ...
INFO: DSC script exported to dsc_result.html
INFO: Constructing DSC from settings_add_method_v1.dsc ...
INFO: Building execution graph & running DSC ...
DSC: 100%|██████████████████████████████████████| 15/15 [00:05<00:00,  2.20it/s]
INFO: Building DSC database ...
INFO: DSC complete!
INFO: Elapsed time 9.648 seconds.


Notice that compared to the run in [Quick Start](Quick_Start.html) which took ~11 seconds, running additional two flavors of the `winsor` module took ~9 seconds. This is because results from `mean` and `median` were skipped in this run; only newly added modules are executed.

Also notice that DSC checks and ensures required package `psych` is available. The package will be installed if it cannot be found on the system.

## Add a New Block
Here we show an alternative style to update the existing DSC file: we use a new syntax block for `winsor` method:

```
mean, median: mean.R, median.R
    x: $x
    $mean: mean

winsor: winsor.R
    x: $x
    trim: 0.1, 0.2
    $mean: mean
```

To run this version,

In [3]:
! dsc settings_add_method_v2.dsc -c 30

INFO: DSC script exported to dsc_result.html
INFO: Constructing DSC from settings_add_method_v2.dsc ...
INFO: Building execution graph & running DSC ...
DSC: 100%|██████████████████████████████████████| 15/15 [00:02<00:00,  5.38it/s]
INFO: Building DSC database ...
INFO: DSC complete!
INFO: Elapsed time 4.885 seconds.


The benchmark is identical to that from previous section albeit different syntax style. Therefore using 5 seconds to check & match file status, DSC2 has skipped all the actual computations.